## Решение задачи бинарной классификации

### Логистическая регрессия

y = {-1, 1}

$b(x) = \sigma(<w,x>)$, где $\sigma(z) = \frac{1}{1 + e^{-z}}$

Логистическая регрессия - это линейный классификатор, который кроме классов умеет предсказывать вероятности классов, а именно, $b(x) = P(y = +1 | x)$

Но об этом на след паре

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
np.random.seed(42)

In [ ]:
data = pd.read_csv('bike_buyers_clean.csv')

In [ ]:
data.head()

# Обзор данных

Проверьте типы колонок в датасете

In [ ]:
# your code here

Колонки с бинарными качественными признаками (типа пола) замените на чиселки

In [ ]:
# your code here

Оставим в данных только числовые колонки.

In [ ]:
# your code here

Поделите данные на матрицу объект-признак и ответ

In [ ]:
# your code here

Проверим сбалансированность классов. Если классы не сбалансированны, мы не можем использовать accuracy

In [ ]:
# your code here

## Масштабирование числовых признаков

In [ ]:
data['Income']

In [ ]:
# Очень большие значения, а у нас есть Пол со значниями 0 и 1...
# Надо масштабировать!

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(data[['Income']])

Важные замечания:
- классы sklearn возвращают numpy arrays, а не pandas dataframe, что неудобно
- любое преобразование признаков в ML-задачах нужно обучать только на тренировочных данных

In [ ]:
# пример работы с пайплайном
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train) # алгоритм находит параметры: mean и sigma. Только к обучающим данным!

X_train_sc = scaler.transform(X_train) # нормирует: x->(x-mean)/sigma
X_test_sc = scaler.transform(X_test) # с помощью mean и sigma от тренировочных данных изменяем test data

In [ ]:
X_train_sc

In [ ]:
X_train = pd.DataFrame(X_train_sc, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_test_sc, index=X_test.index, columns=X_test.columns)

X_train.head()

In [ ]:
y_train.value_counts()

Обучим логистическую регрессию и посмотрим на качество модели.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

# Соберем сделанные преобразования данных в pipeline

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler_', StandardScaler()),
    ('model_', LogisticRegression())
    ])

In [ ]:
# пример работы с пайплайном

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print(accuracy_score(y_test, y_pred))

## Интерпретация результатов

In [ ]:
model.coef_, model.intercept_

In [ ]:
pipe['model_'].coef_, pipe['model_'].intercept_

In [ ]:
coefs = pd.DataFrame(model.coef_, index=np.arange(len(model.coef_)), columns=X_train.columns)
coefs['Intercept'] = model.intercept_
coefs

$$ y = \sigma(0.38*Income - 0.16*Children - 0.53*Cars - 0.1*Age - 0.07) $$

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X, y, cv=3).mean()